In [24]:
import pandas as pd
import ast
from tabulate import tabulate
import re
import numpy as np
import math
import pickle
import html

In [25]:
# Import pickled dictionary of classes

classes_dict = pickle.load(open( "./class_roster_api_dict.pickle", "rb"))
pd_data = pd.DataFrame(classes_dict).transpose()
pd_data = pd_data[["id"]+[col for col in pd_data.columns if col != "id"]]
np_dat = pd_data.to_numpy()

In [26]:
print(np_dat[0])

[353417 {'Banu  Ozer Griffin'} {'TURK 1331'} {'SP18', 'SP19', 'SP20'}
 'Elementary Turkish II'
 'In this course, learners will continue to develop their reading, listening, speaking, and writing beyond the foundation established in Elementary Turkish I. In this semester, learners will advance towards the Intermediate level as well as beginning to feel comfortable and confident getting around in Turkey, undertaking very basic research, and communicating with native Turkish speakers. The format will continue to focus on introducing subjects through receptive skills (reading/listening/watching) with learners then working on projects and longer writing in order to master applicable language.'
 '' 'https://classes.cornell.edu/browse/roster/SP20/class/TURK/1331']


In [27]:
# Import pickled dictionary of RateMyProfessor Data

ratemyprof_dict = pickle.load(open( "./ratemyprofessor_api_dict.pickle", "rb"))

In [28]:
# Example of Rate My Professor Data

gries = ratemyprof_dict["David Joseph Gries"]

# Review for each professor (string)
print(gries["review"])

# Rating for each professor (float)
print(gries["rating"])

# List of tags for each professor (list of strings)
print(gries["tags"])

Gries is incredibly caring and genuinely interested in helping his students. He can be quite sassy on Piazza, which can annoy some students, but I personally thought it was hilarious and gave him personality. Regardless, he is an engaging and humorous lecturer and is very willing to help you if you ask for it.
3.0
['Tough grader', 'Respected', 'Caring', 'Accessible outside class', 'Graded by few things']


In [29]:
#get individual columns as np.arrays

np_ids = np.array(np_dat[:, 0])
np_professors = np.array(np_dat[:, 1])
np_semesters = np.array(np_dat[:,3])
np_subject_number = np.array(np_dat[:, 2])
np_title = np.array(np_dat[:, 4])
np_descriptions = np.array(np_dat[:, 5])
np_outcomes = np.array(np_dat[:, 6])
np_urls = np.array(np_dat[:, 7])


In [30]:
id_to_index = {}

for i in range(len(np_ids)):
    id_to_index[np_ids[i]] = i

In [ ]:
inverted_dict = {}
regex = r'\w+' # regular expression to find words (MAY NEED TO REVISE/EDIT THIS)

    
for i in range(len(np_descriptions)):
#for i in range(len(np_outcomes)):
    #if len(np_outcomes[i]) > 0:
     #   match = ''.join(np_outcomes[i])
    #else:
   #     match = np_descriptions[i]
    print(np_ids[i])
    print(np_subject_number[i])
    print(np_descriptions[i])
    print(np_title[i])
    toks = re.findall(regex, np_descriptions[i].lower())
    seen = set()
    for t in toks:
        if t in inverted_dict.keys():
            if t not in seen:
                count = toks.count(t)
                inverted_dict[t].append((id_to_index[np_ids[i]], count))
                seen.add(t)
        else:
            count = toks.count(t)
            inverted_dict[t] = [(id_to_index[np_ids[i]], count)]
            seen.add(t)
    seen.clear()        



In [ ]:
inverted_dict['the'] #gives list of tuples (doc_index, frequency of 'the' in doc)

In [32]:
min_df = 0
max_df_ratio = 0.17 #tuned this down until it printed out only very common words for a course description
num_docs = np.shape(np_dat)[0]
tokens = set()
idf_dict = {}

for t in inverted_dict.keys():
    df = len(inverted_dict[t])
    if float(df/num_docs) < max_df_ratio:
        tokens.add(t)
        idf_dict[t] = math.log(num_docs/(1 + df), 2)
    else:
        print(t)

in
this
course
will
to
their
and
the
as
with
on
be
a
of
topics
for
students
or
is
an
from
are
we
that
include
how
by


In [33]:
tokens = sorted(tokens)

term_doc_matrix = np.zeros((len(np_descriptions), len(tokens)))

for t in tokens:
    term_idx = tokens.index(t)
    for doc_idx, freq in inverted_dict[t]:
        term_doc_matrix[doc_idx, term_idx] = freq

In [34]:
norms = np.zeros(num_docs)

for i in idf_dict:
    for tup in inverted_dict[i]:
        desc_idx = tup[0]
        desc_term_freq = tup[1]
        norms[desc_idx] += (desc_term_freq * idf_dict[i]) ** 2
        
norms = np.sqrt(norms)

In [15]:
##### THIS WHOLE BLOCK IS COPIED IN THE NEXT ONE DOWN AS A FUNCITON cosine_sim

original_query = "computer programming"

query = original_query.split()
tuples = list()
    
query_norm_sum = 0

for q in query:
    if q in idf_dict.keys():
        q_count = query.count(q)
        q_idf = idf_dict[q]
        query_norm_sum += (q_count*q_idf) ** 2
            
query_norm = math.sqrt(query_norm_sum)
    
doc_scores = {}

for q in query: #iterate over each query term
    if q in idf_dict.keys(): #if q has inverted doc frequency val
        for (doc_idx, value) in inverted_dict[q]: #iterate over each tuple in inverted_index[query_term]
            if doc_idx not in doc_scores.keys():
                doc_scores[doc_idx] = query.count(q) * idf_dict[q] * value * idf_dict[q] #begin accumulator
            else:
                doc_scores[doc_idx] += query.count(q) * idf_dict[q] * value * idf_dict[q]#add to accumulator
            #Additional score for query term in title
            score_boost = 0.1
            if q in np_title[doc_idx].lower():
                doc_scores[doc_idx] += score_boost * (query_norm*norms[doc_idx])
    
    #GET FROM DICT TO LIST OF TUPLES WHILE DIVIDING BY NORMS
    
for doc_idx, value in doc_scores.items():
    tuples.append((value/(query_norm*norms[doc_idx]), doc_idx)) 
        
tuples = sorted(tuples, key=lambda x: x[0], reverse=True)


##### THIS WHOLE BLOCK IS COPIED IN THE NEXT ONE DOWN AS A FUNCITON cosine_sim

In [35]:
def cosine_sim(original_query):
    query = original_query.split()
    tuples = list()
    
    query_norm_sum = 0

    for q in query:
        if q in idf_dict.keys():
            q_count = query.count(q)
            q_idf = idf_dict[q]
            query_norm_sum += (q_count*q_idf) ** 2
            
    query_norm = math.sqrt(query_norm_sum)
    
    doc_scores = {}

    for q in query: #iterate over each query term
        if q in idf_dict.keys(): #if q has inverted doc frequency val
            for (doc_idx, value) in inverted_dict[q]: #iterate over each tuple in inverted_index[query_term]
                if doc_idx not in doc_scores.keys():
                    doc_scores[doc_idx] = query.count(q) * idf_dict[q] * value * idf_dict[q]#begin accumulator
                else:
                    doc_scores[doc_idx] += query.count(q) * idf_dict[q] * value * idf_dict[q]#add to accumulator
                #Additional score for query term in title
                score_boost = 0.1
                if q in np_title[doc_idx].lower():
                    doc_scores[doc_idx] += score_boost * (query_norm*norms[doc_idx])
    
        #GET FROM DICT TO LIST OF TUPLES WHILE DIVIDING BY NORMS
    
    for doc_idx, value in doc_scores.items():
        tuples.append((value/(query_norm*norms[doc_idx]), doc_idx)) 
        
    tuples = sorted(tuples, key=lambda x: x[0], reverse=True)
    return tuples

In [36]:
cosine_sim("finance")

[(0.5221977105343715, 4456),
 (0.5099663461940536, 3956),
 (0.5060845979144261, 1209),
 (0.46344883957306515, 3800),
 (0.45543566116576134, 3797),
 (0.4207575442523396, 3809),
 (0.3889988737098541, 6040),
 (0.3556212098407439, 4709),
 (0.3397716627594753, 953),
 (0.32871275252756316, 4468),
 (0.3222387122890409, 2252),
 (0.3198939025266949, 1194),
 (0.30248998162820423, 6251),
 (0.29822044782250523, 4451),
 (0.2922728923973748, 6253),
 (0.27978390407522274, 946),
 (0.2759119337429835, 6250),
 (0.2733027615544035, 5810),
 (0.26813429203519873, 4749),
 (0.2643970303021992, 4521),
 (0.26308857485557796, 4740),
 (0.2593071306521514, 4718),
 (0.24864368792734445, 6215),
 (0.23648339019571793, 459),
 (0.22986858563679177, 3650),
 (0.22578153018743014, 4466),
 (0.2243334732733384, 2306),
 (0.2238013642207836, 4715),
 (0.22305981066979366, 2248),
 (0.2171117887660779, 1172),
 (0.21312805269690188, 947),
 (0.2113206262978975, 2177),
 (0.20549910539293537, 2278),
 (0.20362659192994176, 3328),
 (

In [37]:
def cosine_sim_class(class_tag): #input is of the form 'INFO 4300' or 'INFO4300'
    
    subject = ("".join(re.split("[^a-zA-Z]*", class_tag))).upper()
    number = str("".join(re.split("[^0-9]*", class_tag)))

    result = [classes_dict[key] for key in classes_dict.keys() 
                 if (subject + " " + number) in classes_dict[key]["subject-number"]][0]
    
    original_query = result["description"]
    
    query = original_query.split()
    tuples = list()
    
    query_norm_sum = 0

    for q in query:
        if q in idf_dict.keys():
            q_count = query.count(q)
            q_idf = idf_dict[q]
            query_norm_sum += (q_count*q_idf) ** 2
            
    query_norm = math.sqrt(query_norm_sum)
    
    doc_scores = {}

    for q in query: #iterate over each query term
        if q in idf_dict.keys(): #if q has inverted doc frequency val
            for (doc_idx, value) in inverted_dict[q]: #iterate over each tuple in inverted_index[query_term]
                if doc_idx not in doc_scores.keys():
                    doc_scores[doc_idx] = query.count(q) * idf_dict[q] * value * idf_dict[q]#begin accumulator
                else:
                    doc_scores[doc_idx] += query.count(q) * idf_dict[q] * value * idf_dict[q] #add to accumulator
                #Additional score for query term in title
                score_boost = 0.1
                if q in np_title[doc_idx].lower():
                    doc_scores[doc_idx] += score_boost * (query_norm*norms[doc_idx])
    
        #GET FROM DICT TO LIST OF TUPLES WHILE DIVIDING BY NORMS
    
    for doc_idx, value in doc_scores.items():
        course = (subject + " " + number)
        if course not in np_subject_number[doc_idx]:
            tuples.append((value/(query_norm*norms[doc_idx]), doc_idx)) 
        
    tuples = sorted(tuples, key=lambda x: x[0], reverse=True)
    #tuples = tuples[1:]
    return tuples

In [38]:
cosine_sim_class("INFO 4300")[0:10]

[(0.9906746652524726, 761),
 (0.9729729913778656, 5393),
 (0.8148851081410464, 4793),
 (0.8128458066152329, 2838),
 (0.8103431886111406, 5629),
 (0.8038798718158042, 6085),
 (0.7985696815257698, 4770),
 (0.7937650812494975, 754),
 (0.787203528197406, 481),
 (0.7811860843118356, 5337)]

In [ ]:
tuples = cosine_sim_class("INFO 4300")

print("#" * len(original_query))
print(original_query)
print("#" * len(original_query))

for score, doc_idx in tuples[:10]:
    print("\n\n")
    print("Score: %s \n" % (score))
    print("Class: %s %s \n" % (" / ".join(np_subject_number[doc_idx]), np_title[doc_idx]))
    print("Description: %s \n" % np_descriptions[doc_idx])
    print("\n\n")

In [ ]:
#original_query = "computer programming"
#tuples = cosine_sim(original_query)

original_query = "finance"

tuples = cosine_sim(original_query)



print("#" * len(original_query))
print(original_query)
print("#" * len(original_query))

for score, doc_idx in tuples[:10]:
    print("\n\n")
    print("Score: %s \n" % (score))
    print("Class: %s %s \n" % (" / ".join(np_subject_number[doc_idx]), np_title[doc_idx]))
    print("Description: %s \n" % np_descriptions[doc_idx])
    print("\n\n")

In [42]:
def rocchio(query, docs, relevant_idx, td_matrix):
    alpha = 1
    beta = 1
    gamma = 1
    
    irr_idx = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
    
    irr_idx = np.setdiff1d(irr_idx, relevant_idx)
    
    num_toks = np.shape(td_matrix)
    
    q_vec = np.zeros(num_toks[1])
    
    q_terms = query.split(' ')
    
    for t in q_terms:
        i = tokens.index(t)
        q_vec[i] = q_vec[i] + 1
        
    term1 = q_vec * alpha
    
    rel_sum = np.zeros(num_toks[1])
    irrel_sum = np.zeros(num_toks[1])
    
    for i in relevant_idx:
        doc = docs[i]
        rel_sum = rel_sum + td_matrix[doc[1], ]

    for j in irr_idx:
        doc = docs[i]
        irrel_sum = irrel_sum + td_matrix[doc[1], ]
        
    term2 = (rel_sum*beta)/len(relevant_idx)
    
    term3 = (irrel_sum*gamma)/len(irr_idx)
    
    new_vec = term1 + term2 - term3
    
    for i in range(len(new_vec)):
        if new_vec[i] < 0:
            new_vec[i] = 0
            
    new_query = ""        
    
    for i in range(len(new_vec)):
        word = tokens[i] + " "
        new_query = new_query + (word * int(round(new_vec[i])))
        
    return new_query

In [48]:
original_query = 'culinary food'

tuples = cosine_sim(original_query)



print("#" * len(original_query))
print(original_query)
print("#" * len(original_query))

for score, doc_idx in tuples[:10]:
    print("*"*100)
    print("Score: %s \n" % (score))
    print("Class: %s %s \n" % (" / ".join(np_subject_number[doc_idx]), np_title[doc_idx]))
    print("Description: %s \n" % np_descriptions[doc_idx])

#############
culinary food
#############
****************************************************************************************************
Score: 0.3992691953681555 

Class: FDSC 6010 Principles and Applications of Food Science and Technology 

Description: Students are first introduced to the principles and applications in food chemistry, food microbiology, food processing and engineering, sensory science, and international food science. Then students review and discuss the research literature in these fields. 

****************************************************************************************************
Score: 0.37178250619610087 

Class: DSOC 3400 Agriculture, Food, Sustainability and Social Justice 

Description: How is our food produced: where, by whom and under what conditions? What are the major trends and drivers of the agriculture and food system? How has our agriculture and food system changed over time? What are some of the environmental, social, nutritional and h

In [50]:
print("Original Query:", original_query)

new_query = rocchio(original_query, tuples[10:], [0, 1, 2, 3], term_doc_matrix)

print("New Query:", new_query)

tuples = cosine_sim(new_query)



print("#" * len(original_query))
print(original_query)
print("#" * len(original_query))

for score, doc_idx in tuples[:10]:
    print("*"*100)
    print("Score: %s \n" % (score))
    print("Class: %s %s \n" % (" / ".join(np_subject_number[doc_idx]), np_title[doc_idx]))
    print("Description: %s \n" % np_descriptions[doc_idx])

Original Query: culinary food
New Query: culinary 
#############
culinary food
#############
****************************************************************************************************
Score: 0.20472560767002818 

Class: PLHRT 1102 Hands-On Horticulture for Gardeners 

Description: The objective is to instill in students a lifelong appreciation for how gardening can enhance individual well-being through aesthetics, culinary experiences, and mastery of techniques. Emphasizes hands-on learning and practice of key gardening skills and techniques in the greenhouse and the field, such as landscape management, garden design, propagation, pruning, grafting, pest management, and flower arrangement. There is one Saturday field trip at the end of the semester to visit gardens in the local area. 

****************************************************************************************************
Score: 0.1910101360054994 

Class: AMST 1886 / HIST 1886 Introduction to Food Studies: Histo

In [22]:
#print professor tags

def professor_tags(class_tag):
    subject = ("".join(re.split("[^a-zA-Z]*", class_tag))).upper()
    number = str("".join(re.split("[^0-9]*", class_tag)))
    result = [classes_dict[key] for key in classes_dict.keys() 
                 if (subject + " " + number) in classes_dict[key]["subject-number"]][0]
    professors = result['professors']
    
    tag_dict = {}
    
    for p in professors:
        tag_dict[p] = ratemyprof_dict[p]['tags']
        
    return tag_dict

In [23]:
tag_dict = professor_tags("INFO3400")
for key,value in tag_dict.items():
    print("%s\n       %s \n" % (key, value))

Alexandra S Hinck
       [] 

Natalie  Bazarova
       ['Amazing lectures', 'Clear grading criteria', 'Would take again', 'Gives good feedback', 'Respected'] 



In [24]:
#SVD for query expansion

from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse.linalg import svds
from sklearn.preprocessing import normalize


In [25]:
vectorizer = TfidfVectorizer(stop_words = 'english', max_df = max_df_ratio,
                            min_df = 0)
my_matrix = vectorizer.fit_transform([x for x in np_descriptions]).transpose()

In [26]:
print(my_matrix.shape)

(21367, 7346)


In [27]:
u, s, v_trans = svds(my_matrix, k=100)
words_compressed, _, docs_compressed = svds(my_matrix, k=40)
docs_compressed = docs_compressed.transpose()

word_to_index = vectorizer.vocabulary_
index_to_word = {i:t for t,i in word_to_index.items()}

words_compressed = normalize(words_compressed, axis = 1)

In [28]:
def closest_words(word_in, k = 10):
    if word_in not in word_to_index: return "Not in vocab."
    sims = words_compressed.dot(words_compressed[word_to_index[word_in],:])
    asort = np.argsort(-sims)[:k+1]
    return [(index_to_word[i],sims[i]/sims[asort[0]]) for i in asort[1:]]

In [29]:
closest_words("linguistics")

[('ascriptions', 0.7823643303771342),
 ('pragmatics', 0.7772622646990378),
 ('syntax', 0.7725482623109073),
 ('plagiarism', 0.7722282028885669),
 ('sociolinguistics', 0.7546739912194449),
 ('sentences', 0.7446880172932401),
 ('resembling', 0.7388748155736673),
 ('phonology', 0.7315319313787894),
 ('semantics', 0.730174836738655),
 ('kunstsprache', 0.7170755033517641)]

In [30]:
closest_words("finance")

[('investing', 0.9618821985185497),
 ('investment', 0.9590164375223831),
 ('valuation', 0.9569130569456731),
 ('investments', 0.9514932285885092),
 ('mergers', 0.9509134389912987),
 ('stock', 0.9464781654345734),
 ('capital', 0.9454415413273359),
 ('debt', 0.9416373766883782),
 ('acquisitions', 0.9409159006529788),
 ('financial', 0.9382709040753279)]